# pip install

In [ ]:
import os
import pandas as pd


In [ ]:
tensorboard_log = '/xx/' # replace with your directory

os.makedirs(tensorboard_log, exist_ok=True)


In [ ]:

df_all_targets = pd.DataFrame()

model_name = 'PPO'


In [ ]:
pip install gymnasium


In [ ]:
pip install "stable-baselines3[extra]>=2.0.0a4"

In [ ]:
import numpy as np

import gymnasium as gym
from gymnasium import spaces

from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.env_util import make_vec_env, SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
from stable_baselines3.common.callbacks import BaseCallback



## set reproducibility

In [ ]:
import torch
import numpy as np
import random
import os
os.environ['PYTHONASHSEED'] = '0'
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# utils functions

In [ ]:
#local modules directory for import
DIR = '/xxhome/' # replace with utils home directory


sys.path.append(DIR)


# import utils fns
from utils.read_data_fns import *
from utils.eval_fns import *

In [ ]:
df_all_targets = pd.DataFrame()
DRL_DIR = '/xx/' # replace with working directory


# clf functions

# Callback functions

# TAREnv class

In [ ]:
#keep track of topics
ALL_VECTORS_PREDICTIONS_DIC = {}
SELECTED_TOPICS_WITHOUT_TARGET = []
ALL_VECORS_PREDICTIONS_DIC_EXIST = False

SELECTED_TOPICS = [] # keep track of all randomly selected topics
SELECTED_TOPICS_TARGET = []

In [ ]:
# import RL env
from rl_utils.ranking_utils import *
from rl_utils.clf_utils import *
from rl_utils.grlstop_tar_env import *


# run experiments

In [ ]:
TRAINING = True

#penalise undershooting, tolerance with overshooting
m = 4
n = 1/4
model_name = 'non_lin'

#penalise overshooting, tolerance with undershooting
m = 1/4
n = 4
model_name = 'non_lin_cost_obj'


#blalanced
m = 1
n = 1
model_name = 'lin_clf_lr'




total_runs = 5 # if choose Deterministic = False (Stochatic) in predict()


target_recalls = [1.0, 0.99, 0.95, 0.9, 0.85, 0.8, 0.75, 0.7]




In [ ]:
df_all_targets = pd.DataFrame()

DRL_DIR = '/xx/' # replace with working directory


## Training

In [ ]:
TRAINING = True
topic_set = 'training'




In [ ]:
training_dataset = 'CLEF'


#### sort topics by target location

In [ ]:
dataset_name = 'CLEF'

qrels = "/data/qrels/CLEF2017_qrels.txt"


qrel_fname, query_rel_dic = load_rel_data(qrels)

run = "/data/rankings/clef2017_training_ranking.txt"

doc_rank_dic, rank_rel_dic, rank_text_dic, rank_tfidf_dic = load_run_data_with_text_tfidf(run, dataset_name, topic_set)

topics_list = make_topics_list(doc_rank_dic,1)

In [ ]:
#remove topic CD008760 last element, contains 64 items only, < 100 vector size
topics_list= topics_list[:-1]


In [ ]:
topics_info = []

for target_recall in target_recalls:
  for t in topics_list:
    topic_id, n_docs, n_rel, prev, target_location = load_topic_target_location(t,target_recall)
    topic_id_target_recall = topic_id + "_" + str(target_recall)
    print(topic_id, n_docs, n_rel, round(prev,3), target_location)
    topics_info.append([topic_id, n_docs, n_rel, prev, target_recall, target_location, topic_id_target_recall])

topics_info

In [ ]:

df = pd.DataFrame(topics_info, columns=['topic_id', 'n_docs', 'n_rel', 'prev', 'target_recall', 'target_location', 'topic_id_target_recall'])
df = df.sort_values(by=['target_recall', 'target_location'])


In [ ]:
sorted_target_loc_topics = list(df['topic_id_target_recall'])


####load clf rel dic if exist

In [ ]:
dataset_name = 'CLEF'
topic_set = 'training'

# file name
ALL_VECTORS_PREDICTIONS_DIC_file_name = dataset_name +'_'+ topic_set +'_'+ 'ALL_VECTORS_PREDICTIONS_DIC.pkl'
ALL_VECTORS_PREDICTIONS_DIC_file_name = DRL_DIR+'data/'+ALL_VECTORS_PREDICTIONS_DIC_file_name

# check if the file exists
if not os.path.exists(ALL_VECTORS_PREDICTIONS_DIC_file_name):
    # if doesn't exist, create an empty dictionary
    ALL_VECTORS_PREDICTIONS_DIC = {}
    SELECTED_TOPICS_WITHOUT_TARGET = []
    ALL_VECTORS_PREDICTIONS_DIC_EXIST = False

else:
    # if exists, load the dictionary from the file
    with open(ALL_VECTORS_PREDICTIONS_DIC_file_name, "rb") as f:
        ALL_VECTORS_PREDICTIONS_DIC = pickle.load(f)

    SELECTED_TOPICS_WITHOUT_TARGET = list(ALL_VECTORS_PREDICTIONS_DIC.keys())
    ALL_VECTORS_PREDICTIONS_DIC_EXIST = True


####ordered topics

In [ ]:
TRAINING = True

SELECTED_TOPICS_ORDERERD = sorted_target_loc_topics
SELECTED_TOPICS_ORDERERD_INDEX = 0

SELECTED_TOPICS_TARGET = target_recalls
SELECTED_TOPICS_TARGET_INDEX = 0

# Instantiate the vec env

#random topic selection for each env instance
SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics
SELECTED_TOPICS_TARGET = []

n_envs=len(sorted_target_loc_topics)
vec_env = make_vec_env(TAREnv, n_envs=len(sorted_target_loc_topics), env_kwargs=dict(target_recall=None, topics_list = sorted_target_loc_topics, topic_id=None, size=100, render_mode='human'))

SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics
SELECTED_TOPICS_TARGET = []

train_size = len(topics_list)*len(target_recalls)


vec_env_train = vec_env

#save dic for first time
if not ALL_VECTORS_PREDICTIONS_DIC_EXIST:
    # Save the dictionary to a file
    with open(ALL_VECTORS_PREDICTIONS_DIC_file_name, 'wb') as f:
        pickle.dump(ALL_VECTORS_PREDICTIONS_DIC, f)

#### Hyperparameter Settings

In [ ]:
def nearest_factor_of(target, total):
    # find all factors of `total`
    factors = [i for i in range(1, total + 1) if total % i == 0]

    # find the factor closest to the target
    closest_factor = min(factors, key=lambda x: abs(x - target))
    return closest_factor



In [ ]:


# Train the agent

n_envs=len(sorted_target_loc_topics)


learning_rate_initial = 0.0003


learning_rate = learning_rate_initial
learning_rate_type = '_lr_static'


ent_coef = 0.1

gamma = 0.99
gae_lambda = 0.98
clip_range=0.1




n_epochs =10

n_steps = 10



total_rollout = n_steps * n_envs  # Total rollout buffer size
target_mini_batch = int(1/4 * total_rollout)  # Target value (1/4 of total)


# get the nearest factor
batch_size = nearest_factor_of(total_rollout, target_mini_batch)




nn_nodes = 64

policy_kwargs = dict(
    net_arch=[nn_nodes, nn_nodes]  # Two hidden layers with nn_nodes
)


total_timesteps = 20_000_000


####  PPO

In [ ]:


tb_log_name = model_name+"_"+training_dataset+"_nstps"+str(n_steps)+"_btch"+str(batch_size)+"_ts"+str(total_timesteps)+ "_ent"+str(ent_coef)+"_epch"+str(n_epochs)+learning_rate_type+str(learning_rate_initial)+"_clip"+str(clip_range)+"_nn"+str(nn_nodes) +"_target"+str(target_recall)


callback_log = tensorboard_log+'running/'
# Checkpoint callback to save the model periodically
checkpoint_callback = CheckpointCallback(save_freq=10000, save_path=callback_log, name_prefix=tb_log_name)



# patience is set to patience steps
patience = 10*n_steps*n_envs #as a general early stopping rule, allow for 10 rollouts w/o improvement more than 1%

early_stopping_callback = EarlyStoppingCallback(patience=patience, min_delta=0.001, verbose=1)



# Combine the callbacks
callbacks = [checkpoint_callback, early_stopping_callback]

model = PPO(
    policy = 'MlpPolicy',
    env = vec_env_train,
    n_steps = n_steps,
    batch_size = batch_size,
    n_epochs = n_epochs,
    gamma = gamma,
    gae_lambda = gae_lambda,
    ent_coef = ent_coef,
    clip_range = clip_range,
    verbose=0,
    learning_rate = learning_rate,
    policy_kwargs=policy_kwargs,
    seed=0,
    tensorboard_log= tensorboard_log)




model.learn(total_timesteps=total_timesteps, tb_log_name=tb_log_name, callback=callbacks)

model.save(tensorboard_log+'model_'+tb_log_name)



In [ ]:
%load_ext tensorboard
%tensorboard --logdir "$tensorboard_log"


## TESTING

### Load Trained Model

In [ ]:
TRAINING = False


model_load_dir = tensorboard_log + 'model_'+tb_log_name


model = PPO.load(model_load_dir, env=vec_env_train)

### all targets & datasets

In [ ]:


dataset_names = ["CLEF2017", "CLEF2018", "CLEF2019"]

qrels_dic = {
    "CLEF2017" : "data/qrels/CLEF2017_qrels.txt",
    "CLEF2018" : "data/qrels/CLEF2018_qrels.txt",
    "CLEF2019" : "data/qrels/CLEF2019_qrels.txt",
}
rankings_dic = {
    "CLEF2017" : "data/rankings/clef2017_ranking.txt",
    "CLEF2018" : "data/rankings/clef2018_ranking.txt",
    "CLEF2019" : "data/rankings/clef2019_ranking.txt",
}


target_recalls = [1.0, 0.9, 0.8, 0.7]


### results

In [ ]:
TRAINING = False

total_runs = 1
topic_set = 'test'




### all targets & datasets
for dataset_name in dataset_names:

  # file name
  ALL_VECTORS_PREDICTIONS_DIC_file_name = dataset_name +'_'+ topic_set +'_'+ 'ALL_VECTORS_PREDICTIONS_DIC.pkl'
  ALL_VECTORS_PREDICTIONS_DIC_file_name = DRL_DIR+'data/'+ALL_VECTORS_PREDICTIONS_DIC_file_name

  # check if the file exists
  if not os.path.exists(ALL_VECTORS_PREDICTIONS_DIC_file_name):
    # if doesn't exist, create an empty dictionary
    ALL_VECTORS_PREDICTIONS_DIC = {}
    SELECTED_TOPICS_WITHOUT_TARGET = []
    ALL_VECTORS_PREDICTIONS_DIC_EXIST = False
  else:
    # if exists, load the dictionary from the file
    with open(ALL_VECTORS_PREDICTIONS_DIC_file_name, "rb") as f:
      ALL_VECTORS_PREDICTIONS_DIC = pickle.load(f)
      SELECTED_TOPICS_WITHOUT_TARGET = list(ALL_VECTORS_PREDICTIONS_DIC.keys())
      ALL_VECTORS_PREDICTIONS_DIC_EXIST = True

  qrels = qrels_dic[dataset_name]

  qrel_fname, query_rel_dic = load_rel_data(qrels)

  run = rankings_dic[dataset_name]
  #doc_rank_dic, rank_rel_dic = load_run_data(run)
  doc_rank_dic, rank_rel_dic, rank_text_dic, rank_tfidf_dic = load_run_data_with_text_tfidf(run, dataset_name, topic_set)

  topics_list = make_topics_list(doc_rank_dic,1)  # sort topics by no docs

  if dataset_name == 'CLEF2019':
      #remove topic CD012164 last element, contains 61 items only, < 100 vector size
      topics_list= topics_list[:-1]

  for target_recall in target_recalls:

    # Instantiate the vec env

    #random topic selection for each env instance
    SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics

    vec_env = make_vec_env(TAREnv, n_envs=len(topics_list), env_kwargs=dict(target_recall=target_recall, topics_list = topics_list, topic_id=None, size=100, render_mode='human'))

    SELECTED_TOPICS = [] # reset before/after each call, keep track of all randomly selected topics

    test_size = len(topics_list)
    vec_env_test = vec_env

    df = pd.DataFrame()
    df_all_runs = pd.DataFrame()

    for run in range(total_runs):

      # Test the trained agent
      # using the vecenv
      vec_env_test = vec_env
      obs = vec_env_test.reset()
      test_steps = 100
      vector_size = 100


      n_env = test_size
      agent=0
      target=0
      agent_vector=[]
      terminal_observation=[]

      topics = []
      recalls = []
      costs=[]
      e_costs = []
      reliabilities = []
      rewards = []
      distances = []
      differences = []
      targets = []
      run_cnts = []

      for eID in range(test_size):
        print(f"=================== env {eID}")
        env = vec_env_test.envs[eID]
        obs, info = env.reset()

        for step in range(test_steps):

          action, _ = model.predict(obs, deterministic=True) # predict all next steps

          obs, reward, done, trun,info = env.step(action)

          if done or trun:
                      topic_id = info['topic_id']
                      recall = info['recall']
                      cost = info['cost']
                      e_cost =  info['e_cost']

                      distance = info['distance']

                      agent = info['agent']
                      target = info['target']
                      agent_vector = info['agent_vector']
                      terminal_observation = info['terminal_observation']

                      difference = target_recall - recall



                      reliability = 1 if recall >= target_recall else 0
                      topics.append(topic_id)
                      recalls.append(recall)
                      costs.append(cost)
                      e_costs.append(e_cost)
                      reliabilities.append(reliability)
                      rewards.append(reward)
                      distances.append(distance)
                      targets.append(target)
                      run_cnts.append(run)
                      differences.append(difference)

                      df_tmp = pd.DataFrame( list(zip([dataset_name]*len(topics_list), topics, run_cnts, recalls, reliabilities, costs, e_costs, rewards, differences, distances, targets)),
                      columns =['Dataset', 'Topic', 'Run', 'Recall', 'Reliability', 'Cost', 'e-Cost', 'Reward', 'Difference', 'Distance', 'Target'])


                      df = pd.concat([df_tmp])


                      df.groupby('Topic').mean(numeric_only=True)

                      break



      df_all_runs = pd.concat([df_all_runs, df])



    df_all_runs['Model'] = model_name
    df_all_runs['Model_settings'] = tb_log_name
    df_all_runs['Target_Recall'] = target_recall



    df_all_targets = pd.concat([df_all_targets, df_all_runs], ignore_index = True)

    #save dic for first time
    if not ALL_VECTORS_PREDICTIONS_DIC_EXIST:
        # Save the dictionary to a file
        with open(ALL_VECTORS_PREDICTIONS_DIC_file_name, 'wb') as f:
            pickle.dump(ALL_VECTORS_PREDICTIONS_DIC, f)


display(df_all_targets)

display(df_all_targets.describe())

display(df_all_targets.groupby(['Target_Recall','Dataset']).mean(numeric_only=True).round(3))
display(df_all_targets.groupby(['Target_Recall','Dataset']).std(numeric_only=True).round(3))


